In [ ]:
import nbformat

notebook_path = "HyPE.ipynb"

nb = nbformat.read(notebook_path, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

nbformat.write(nb, notebook_path)

print("Fixed!")


In [ ]:
!pip install langchain-community langchain-text-splitters langchain-ollama pypdf pymupdf langchain-huggingface sentence-transformers langchain-experimental faiss-cpu  --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 102.8 MB/s eta 0:00:00


In [ ]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 19.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.3.7 which is incompatible.


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List
from concurrent.futures import ThreadPoolExecutor,as_completed
from tqdm import tqdm
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
import faiss
from langchain_core.documents import Document

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

model_id = "Qwen/Qwen3-4B-Instruct-2507"

# ==========================================
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 確保 pipeline 正常運作的常見修復
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# ==========================================
# 建立原生 Transformers Pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,      # 決定回答長度
    temperature=0.2,         # RAG 建議設低一點，減少幻覺
    repetition_penalty=1.1,  # 避免鬼打牆
    return_full_text=False   # 重要：設為 False 只回傳生成的部分
)

# . 轉為 LangChain 的 LLM 物件
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# (關鍵) 轉為 ChatModel 物件
# ChatHuggingFace 會自動處理 prompt template (User/System/Assistant 格式)
chat_model = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'temperature', 'repetition_penalty', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [ ]:
embedding_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cuda'},
)

Loading weights:   0%|          | 0/310 [00:00<?, ?it/s]

In [ ]:
path=r"/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf"
Chunk_size=512
Chunk_overlap=0

question="What the CRAG proposed to improve "

In [ ]:
def retriever_context_per_question(question,query_retriever):

    docs=query_retriever.invoke(question,k=2)

    context=[doc.page_content for doc in docs]

    return context,docs

def replace_tab_with_space(texts):
    for text in texts:
        text.page_content=text.page_content.replace("\t"," ")
    return texts

def show_context(context):
    """
    Display the contents of the provided context list.

    Args: context (list): A list of context items to be displayed.

    Prints each context item in the list with a heading indicating its position.
    """
    for i, c in enumerate(context):
        print(f"Context {i + 1}:")
        print(c)
        print("\n")

In [ ]:
#一個chunk生成多個問題，每個問題做embedding
def generate_hypothetical_prompt_embeddings(chunk_text: str):
    """
    Parameters:
    chunk_text (str): Text contents of the chunk

    Returns:
    chunk_text (str): Text contents of the chunk. This is done to make the
        multithreading easier
    hypothetical prompt embeddings (List[float]): A list of embedding vectors
        generated from the questions
    """



    template="""
    Analyze the input text and generate essential questions that, when answered,
    capture the main points of the text. Each question should be one line,
    without numbering or prefixes.\n\n
    Text:\n<chunk_text>{chunk_text}</chunk_text>\n\n
    Questions:\n
    """
    prompt=PromptTemplate.from_template(template)

    question_chain=prompt | llm | StrOutputParser()

    question=question_chain.invoke({"chunk_text":chunk_text}).replace("\n\n","\n").split("\n")

    return chunk_text,embedding_model.embed_documents(question)

#一個問題有多個embedding vector,每個(問題,vector)都加入vectorstore


def prepare_vectorstore(documents_list):
    """
    修正後的函數，專門接收 List[Document]
    """
    # 儲存結構：[(chunk_text, embedding_vector), ...]
    text_embedding_pairs = []

    # 用來保留 metadata (例如頁碼)
    metadatas = []

    print(f"Processing {len(documents_list)} chunks...")

    for doc in tqdm(documents_list):
        try:
            # 🛑 修正點 1：這裡是 Document 物件，必須取出 .page_content 才能當字串用
            original_text = doc.page_content
            original_metadata = doc.metadata  # 保留原始 metadata

            # 1. 呼叫生成函數 (傳入純字串)
            # 這裡假設你的 generate_hypothetical_prompt_embeddings 接收字串並回傳 (text, vectors)
            _, question_vectors = generate_hypothetical_prompt_embeddings(original_text)

            # 2. 處理回傳的向量
            for vec in question_vectors:
                # 安全性檢查：轉為 List
                if hasattr(vec, 'tolist'):
                    vec = vec.tolist()
                elif hasattr(vec, 'cpu'):
                    vec = vec.detach().cpu().numpy().tolist()

                # 3. 收集數據： HyPE 核心邏輯
                # 讓「問題向量」對應到「原文內容」
                text_embedding_pairs.append((original_text, vec))
                metadatas.append(original_metadata)

        except Exception as e:
            print(f"Error processing a chunk: {e}")
            continue

    if not text_embedding_pairs:
        return None

    # 4. 建立 FAISS 索引
    print(f"Indexing {len(text_embedding_pairs)} vectors...")

    # 🛑 修正點 2：使用 from_embeddings
    vectorstore = FAISS.from_embeddings(
        text_embeddings=text_embedding_pairs,
        embedding=embedding_model,
        metadatas=metadatas # 把正確的 metadata 塞回去
    )

    return vectorstore

def encode_PDF(path,chunk_size,chunk_overlap):
    loader=PyPDFLoader(path)
    documents=loader.load()

    texts_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    texts=texts_splitter.split_documents(documents)

    cleaned_texts=replace_tab_with_space(texts)

    vectorstore = prepare_vectorstore(cleaned_texts)

    return vectorstore


In [ ]:
vectorstore=encode_PDF(path,Chunk_size,Chunk_overlap)
retriever=vectorstore.as_retriever()




Processing 41 chunks...


100%|██████████| 41/41 [05:56<00:00,  8.70s/it]

Indexing 292 vectors...


In [ ]:
# 你的 retriever 已經存在記憶體中了，不需要重新建立
# 重新執行檢索與展示的部分即可

# 1. 重新檢索
context_list, source_docs = retriever_context_per_question(question, retriever)

# 2. 修正去重邏輯 (針對純文字列表去重)
unique_context = list(set(context_list))

# 3. 顯示結果
print(f"成功檢索到 {len(unique_context)} 個段落：\n")
show_context(unique_context)

成功檢索到 2 個段落：

Context 1:
instance in practice, as detailed in Table 6. The
findings indicate that the self-correction mecha-
nism incurs only modest computational overhead
while significantly enhancing performance, thereby
validating its lightweight nature.
6 Conclusion & Limitation
This paper studies the problem where RAG-based
approaches are challenged if retrieval goes wrong,
thereby exposing inaccurate and misleading knowl-
edge to generative LMs. Corrective Retrieval
Augmented Generation is proposed to improve the
robustness of generation. Essentially, a lightweight
retrieval evaluator is to estimate and trigger three
knowledge retrieval actions discriminately. With
the further leverage of web search and optimized
knowledge utilization, CRAG has significantly
improved the ability of automatic self-correction
and efficient utilization of retrieved documents.
Experiments extensively demonstrate its adaptabil-
ity to RAG-based approaches as well as general-
izability across short- an